# Example: Turbofan Engine Degradation Simulation Data Set

This uses the dataset explored in Dayne Batten's implementation of WTTE-RNN in Keras.

Reference:
```
A. Saxena and K. Goebel (2008). 
"Turbofan Engine Degradation Simulation Data Set", 
https://ti.arc.nasa.gov/c/13/, NASA Ames, Moffett Field, CA.
```

In [ ]:
from wtte.network import WtteRnnNetwork
from wtte.loss import loss_continuous_weibull_loglik, loss_discrete_weibull_loglik
from wtte.datasets import TurbofanDegradationDataset
from wtte.train import train

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
dataset_train = TurbofanDegradationDataset(directory='D:/Users/Aaron/Documents/AI and ML Projects/Data/CMAPSS',
                                           train=True, min_seq_len=20, max_seq_len=100,
                                           unit_ids=[1])

logging.info(len(dataset_train))
#dataset_train.df.head()

In [ ]:
dataset_test = TurbofanDegradationDataset(directory='D:/Users/Aaron/Documents/AI and ML Projects/Data/CMAPSS',
                                           train=False, min_seq_len=100, max_seq_len=100,
                                           unit_ids=[1])
dataset_test.standardize(dataset_train)

logging.info(len(dataset_test))
#dataset_test.df.head()

In [ ]:
dl_train = DataLoader(dataset_train, batch_size=2048, shuffle=True, num_workers=12, pin_memory=True,
                      collate_fn=dataset_train.collate_fn)

In [ ]:
dl_test = DataLoader(dataset_test, batch_size=2048, shuffle=False, num_workers=12, pin_memory=True,
                      collate_fn=dataset_train.collate_fn)

In [ ]:
model = WtteRnnNetwork(input_size=len(dataset_train.features), 
                       rnn_layer=nn.GRU, 
                       rnn_layer_options={'hidden_size': 50, 'num_layers': 1, 'dropout': 0.25},
                       init_alpha=1.0, max_beta_value=10.0) \
                           .to(device)

In [ ]:
train(model, dl_train, test_dataloader=dl_test, n_epochs=100, lr=0.01, clip_grad=1.0, 
      loss_type='discrete', device=device, pretrain=True)

In [ ]:
# Save trained model
torch.save(model.state_dict(), 'D:/Users/Aaron/Documents/AI and ML Projects/My Projects/Predictive Maintenance/wtte_rnn.pt')